<a href="https://colab.research.google.com/github/rajy4683/EVA4/blob/master/CPPBuillderColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This Notebook allows you to use Libtorch C++ Frontend APIs on Google Colab.
##### It will allow you to create required C++ files (.cpp/.h) in individual cells and also compile and execute them in the Colab framework.
The model used in this notebook is taken EVA4 course offered by
 **"The School of AI"** 
  and our awesome mentor [**Rohan Shravan**](https://in.linkedin.com/in/rohanshravan).
###### I will create a medium article shortly listing the basic setup steps.
###### Please feel free to comment on my github link for any improvements.


In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import os
import subprocess
import tempfile
import uuid

from IPython.core.magic import Magics, cell_magic, magics_class
from IPython.core.magic_arguments import argument, magic_arguments, parse_argstring


In [20]:
%cd /content

/content


In [24]:
!nvidia-smi

Tue Jan 28 11:34:42 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.44       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

#### Download the prebuilt C++ libtorch from pytorch.org
#### Current colab instance has cudatoolkit 10.1. To check the correct version use:


```
!nvidia-smi
```


##### Details on stand-alone compilation are available at [https://pytorch.org/cppdocs/installing.html](https://pytorch.org/cppdocs/installing.html)


In [21]:
!wget https://download.pytorch.org/libtorch/cu101/libtorch-cxx11-abi-shared-with-deps-1.4.0.zip 

--2020-01-28 10:49:43--  https://download.pytorch.org/libtorch/cu101/libtorch-cxx11-abi-shared-with-deps-1.4.0.zip
Resolving download.pytorch.org (download.pytorch.org)... 13.224.29.60, 13.224.29.19, 13.224.29.73, ...
Connecting to download.pytorch.org (download.pytorch.org)|13.224.29.60|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 762860883 (728M) [application/zip]
Saving to: ‘libtorch-cxx11-abi-shared-with-deps-1.4.0.zip’

libtorch-cxx11-abi- 100%[===================>] 727.52M  52.1MB/s    in 13s     

2020-01-28 10:49:57 (56.2 MB/s) - ‘libtorch-cxx11-abi-shared-with-deps-1.4.0.zip’ saved [762860883/762860883]



In [22]:
!unzip libtorch-cxx11-abi-shared-with-deps-1.4.0.zip 

Archive:  libtorch-cxx11-abi-shared-with-deps-1.4.0.zip
   creating: libtorch/
 extracting: libtorch/build-version  
   creating: libtorch/include/
   creating: libtorch/include/pybind11/
  inflating: libtorch/include/pybind11/common.h  
  inflating: libtorch/include/pybind11/attr.h  
  inflating: libtorch/include/pybind11/pybind11.h  
  inflating: libtorch/include/pybind11/eval.h  
  inflating: libtorch/include/pybind11/functional.h  
  inflating: libtorch/include/pybind11/stl_bind.h  
  inflating: libtorch/include/pybind11/iostream.h  
  inflating: libtorch/include/pybind11/eigen.h  
  inflating: libtorch/include/pybind11/stl.h  
  inflating: libtorch/include/pybind11/operators.h  
  inflating: libtorch/include/pybind11/buffer_info.h  
  inflating: libtorch/include/pybind11/complex.h  
  inflating: libtorch/include/pybind11/embed.h  
   creating: libtorch/include/pybind11/detail/
  inflating: libtorch/include/pybind11/detail/common.h  
  inflating: libtorch/include/pybind11/detail/in

#### Save the "libtorch" folder into Google Drive to avoid losing the content upon Colab restarts

In [0]:
!mv /content/libtorch /content/gdrive/My\ Drive/

In [0]:
#### needed as cmake sometimes fails to understand the space in "My Drive" :(
!ln -sf /content/gdrive/My\ Drive/libtorch /content/libtorch2

#### Copy "example-app" folder from github to any folder of your choice. 
For this notebook the example-app in present in Google Drive path:
#####   "/content/gdrive/My Drive/example-app"
Folder structure should be as follow:

    example-app/
       |-CMakeLists.txt
       |-src/
       |-|--example-app.cpp
       |-include/
       |-|--model_includes.h
       |-build/


In [0]:
!cp -R /content/EVA4CPPNotebook/example-app /content/gdrive/My\ Drive/example-app

#### Below cell will setup the required build environment. 
You might see multiple files created in build directory.
#### Current CMakeLists.txt:

```
cmake_minimum_required(VERSION 3.0 FATAL_ERROR)
project(example-app)

find_package(Torch REQUIRED)
#set()
file(GLOB INCLUDES_DIR ${PROJECT_SOURCE_DIR}/include/*.h)
file(GLOB SOURCE_DIR ${PROJECT_SOURCE_DIR}/src/*.cpp)

message("Project directory ${PROJECT_SOURCE_DIR}")
#add_subdirectory(src)
include_directories(${PROJECT_SOURCE_DIR}/include ${PROJECT_SOURCE_DIR}/src)
set(CMAKE_CXX_FLAGS "${CMAKE_CXX_FLAGS} ${TORCH_CXX_FLAGS}")

add_executable(example-app ${INCLUDES_DIR} ${SOURCE_DIR} )

target_link_libraries(example-app "${TORCH_LIBRARIES}")
set_property(TARGET example-app PROPERTY CXX_STANDARD 14)

# The following code block is suggested to be used on Windows.
# According to https://github.com/pytorch/pytorch/issues/25457,
# the DLLs need to be copied to avoid memory errors.
if (MSVC)
  file(GLOB TORCH_DLLS "${TORCH_INSTALL_PREFIX}/lib/*.dll")
  add_custom_command(TARGET example-app
                     POST_BUILD
                     COMMAND ${CMAKE_COMMAND} -E copy_if_different
                     ${TORCH_DLLS}
                     $<TARGET_FILE_DIR:example-app>)
endif (MSVC)
```



In [0]:
%cd /content/gdrive/My\ Drive/example-app/build
!cmake -DCMAKE_PREFIX_PATH=/content/libtorch2 ..
!cmake --build . --config Release

In [0]:
'''
    Below plugin is modified version of https://github.com/andreinechaev/nvcc4jupyter 
    "cuda" method has been modified to create project directory and store 
    .cpp files in project_dir/src and .h files in project_dir/include.
    Use %%cuda <arguments> at the start of a cell to create a .cpp/.h file  
'''
import argparse

def get_argparser():
    parser = argparse.ArgumentParser(description='CCPlugin params')
    parser.add_argument("-t", "--timeit", action='store_true',
                        help='flag to return timeit result instead of stdout')
    return parser


def print_out(out: str):
    for l in out.split('\n'):
        print(l)

@magics_class
class CCPlugin(Magics):

    def __init__(self, shell):
        super(CCPlugin, self).__init__(shell)
        self.argparser = get_argparser()
        current_dir = os.getcwd()
        self.output_dir = os.path.join(current_dir)#, 'parent_dir')
        #if not os.path.exists(self.output_dir):
        #    os.mkdir(self.output_dir)
        print(f'Default output directory at {self.output_dir}')
        #else:
        #    print(f'directory {self.output_dir} already exists')

        self.out = os.path.join(current_dir, "result.out")
        print(f'Cpp files outpath {self.out}')


    @magic_arguments()
    @argument('-p', '--parent_dir', type=str, help='Parent directory for the application', nargs='+')
    @argument('-n', '--name', type=str, help='File name. Can be .h or .cpp files')
    @argument('-c', '--compile', type=bool, help='Should be compiled?')
    @cell_magic
    def cuda(self, line='', cell=None):
        args = parse_argstring(self.cuda, line)
        ex = args.name.split('.')[-1]
        parent_dir = ' '.join(args.parent_dir)
        final_path = ""
        print(parent_dir)

        if ex not in ['cpp', 'h']:   
            raise Exception('name must end with .cpp or .h')
        if parent_dir is not None:
            print(parent_dir)
            self.output_dir = parent_dir           
                               
        if not os.path.exists(self.output_dir):
            print(f'Output directory does not exist, creating')
            try:
                os.mkdir(self.output_dir)
            except OSError:
                print(f"Creation of the directory {self.output_dir} failed")
            else:
                print(f"Successfully created the directory {self.output_dir}")
        if ex == 'h':
            self.output_dir = os.path.join(self.output_dir,'include')
        else:
            self.output_dir = os.path.join(self.output_dir,'src')
        

        file_path = os.path.join(self.output_dir, args.name)

        with open(file_path, "w") as f:
            f.write(cell)
def load_ipython_extension(ipython):
    """
    Any module file that define a function named `load_ipython_extension`
    can be loaded via `%load_ext module.path` or be configured to be
    autoloaded by IPython at startup time.
    """
    # You can register the class itself without instantiating it.  IPython will
    # call the default constructor on it.
    ipython.register_magics(CCPlugin(ipython))

In [8]:
load_ipython_extension(get_ipython())

Default output directory at /content
Cpp files outpath /content/result.out


In [34]:
%%cuda -p /content/gdrive/My Drive/cpp_dl/example-app -n model_includes.h
#include <torch/torch.h>
#include <iostream>

struct TestNet : torch::nn::Module {
  TestNet(int64_t N, int64_t M)
      : linear(register_module("linear", torch::nn::Linear(N, M))) {
    another_bias = register_parameter("b", torch::randn(M));
  }
  torch::Tensor forward(torch::Tensor input) {
    return linear(input) + another_bias;
  }
  torch::nn::Linear linear;
  torch::Tensor another_bias;
};

struct EVA4BasicConv: torch::nn::Module {
  EVA4BasicConv():
        conv1(torch::nn::Conv2dOptions(1, 32, 3).padding(1).bias(false)),     
        conv2(torch::nn::Conv2dOptions(32, 64, 3).padding(1).bias(false)),
        pool1(torch::nn::MaxPool2dOptions(2).stride(2)),    
        conv3(torch::nn::Conv2dOptions(64, 128, 3).padding(1).bias(false)),  
        conv4(torch::nn::Conv2dOptions(128, 256, 3).padding(1).bias(false)),
        pool2(torch::nn::MaxPool2dOptions(2).stride(2)), 
        conv5(torch::nn::Conv2dOptions(256, 512, 3).padding(0).bias(false)),
        conv6(torch::nn::Conv2dOptions(512, 1024, 3).padding(0).bias(false)),
        conv7(torch::nn::Conv2dOptions(1024, 10, 3).padding(0).bias(false))

    {
        // register_module() is needed if we want to use the parameters() method later on
        register_module("conv1", conv1);
        register_module("conv2", conv2);
        register_module("conv3", conv3);
        register_module("conv4", conv4);
        register_module("conv5", conv5);
        register_module("conv6", conv6);
        register_module("conv7", conv7);
        register_module("pool1", pool1);
        register_module("pool2", pool2);
    }
    torch::Tensor forward(torch::Tensor x) {
        x = pool1(torch::relu(conv2->forward(torch::relu(conv1->forward(x)))));
        x = pool2(torch::relu(conv4->forward(torch::relu(conv3->forward(x)))));
        x = torch::relu(conv6->forward(torch::relu(conv5->forward(x))));
        x = torch::relu(conv7->forward(x));
        x = x.view({-1,10});
        x = torch::nn::functional::detail::log_softmax(x, 1, c10::nullopt);
        //x = torch::log_softmax(x, 10);
        return x;
     }
    torch::nn::Conv2d conv1, conv2, conv3, conv4,conv5, conv6, conv7;
    torch::nn::MaxPool2d pool1, pool2;
 //nn::BatchNorm2d batch_norm1, batch_norm2, batch_norm3;
};

//############## const globals #################
// Mostly taken from https://github.com/pytorch/examples/tree/master/cpp/mnist
// The batch size for testing and training.
const int64_t kTestBatchSize = 1000;
const int64_t kTrainBatchSize = 128;

// The number of epochs to train.
const int64_t kNumberOfEpochs = 10;

// After how many batches to log a new update with the loss value.
const int64_t kLogInterval = 10; 

// Path where MNIST dataset will be downloaded.
// ########## EDIT THIS PATH AS PER your directory structure or location where 
// ########## YOU WANT TO STORE THE MNIST dataset
const char* kDataFolder = "/content/gdrive/My Drive/cpp_dl/MNIST/raw";


/content/gdrive/My Drive/cpp_dl/example-app
/content/gdrive/My Drive/cpp_dl/example-app


In [37]:
%%cuda -p /content/gdrive/My Drive/cpp_dl/example-app -n example-app.cpp
#include <torch/torch.h>
#include <iostream>
#include "model_includes.h" // The header file created above is included here
using namespace torch;


// Training method
template <typename DataLoader>
void train(
    size_t epoch,
    EVA4BasicConv& model,
    torch::Device device,
    DataLoader& data_loader,
    torch::optim::Optimizer& optimizer,
    size_t dataset_size) {
  model.train();
  size_t batch_idx = 0;
  for (auto& batch : data_loader) {
    auto data = batch.data.to(device), targets = batch.target.to(device);
    optimizer.zero_grad();
    auto output = model.forward(data);
    auto loss = torch::nll_loss(output, targets);
    AT_ASSERT(!std::isnan(loss.template item<float>()));
    loss.backward();
    optimizer.step();

    if (batch_idx++ % kLogInterval == 0) {
      std::printf(
          "\rTrain Epoch: %ld [%5ld/%5ld] Loss: %.4f",
          epoch,
          batch_idx * batch.data.size(0),
          dataset_size,
          loss.template item<float>());
    }
  }
}

// Test method
template <typename DataLoader>
void test(
    EVA4BasicConv& model,
    torch::Device device,
    DataLoader& data_loader,
    size_t dataset_size) {
  torch::NoGradGuard no_grad;
  model.eval();
  double test_loss = 0;
  int32_t correct = 0;
  for (const auto& batch : data_loader) {
    auto data = batch.data.to(device), targets = batch.target.to(device);
    auto output = model.forward(data);
    test_loss += torch::nll_loss(
                     output,
                     targets,
                     /*weight=*/{},
                     torch::Reduction::Sum)
                     .template item<float>();
    auto pred = output.argmax(1);
    correct += pred.eq(targets).sum().template item<int64_t>();
  }

  test_loss /= dataset_size;
  std::printf(
      "\nTest set: Average loss: %.4f | Accuracy: %.3f\n",
      test_loss,
      static_cast<double>(correct) / dataset_size);
}

auto main() -> int {
  torch::manual_seed(1);
  
  torch::DeviceType device_type;
  if (torch::cuda::is_available()) {
    std::cout << "CUDA available! Training on GPU." << std::endl;
    device_type = torch::kCUDA;
  } else {
    std::cout << "Training on CPU." << std::endl;
    device_type = torch::kCPU;
  }
  torch::Device device(device_type);

  EVA4BasicConv model;
  model.to(device);
  std::cout << "Initialize training and test dataloaders" <<std::endl;
  auto train_dataset = torch::data::datasets::MNIST(kDataFolder)
                           .map(torch::data::transforms::Normalize<>(0.1307, 0.3081))
                           .map(torch::data::transforms::Stack<>());
  const size_t train_dataset_size = train_dataset.size().value();
  auto train_loader =
      torch::data::make_data_loader<torch::data::samplers::SequentialSampler>(
          std::move(train_dataset), kTrainBatchSize);

  auto test_dataset = torch::data::datasets::MNIST(
                          kDataFolder, torch::data::datasets::MNIST::Mode::kTest)
                          .map(torch::data::transforms::Normalize<>(0.1307, 0.3081))
                          .map(torch::data::transforms::Stack<>());
  const size_t test_dataset_size = test_dataset.size().value();

  std::cout<< "Size of Training DataSet:" << train_dataset_size <<std::endl;
  std::cout<< "Size of Test DataSet:" << test_dataset_size <<std::endl;
  auto test_loader =
      torch::data::make_data_loader(std::move(test_dataset), kTestBatchSize);

  torch::optim::SGD optimizer(
      model.parameters(), torch::optim::SGDOptions(0.01).momentum(0.9));
  std::cout << "Starting training+test cycle" <<std::endl;    
  for (size_t epoch = 1; epoch <= kNumberOfEpochs; ++epoch) {
    train(epoch, model, device, *train_loader, optimizer, train_dataset_size);
    test(model, device, *test_loader, test_dataset_size);
  }
}

/content/gdrive/My Drive/cpp_dl/example-app
/content/gdrive/My Drive/cpp_dl/example-app


In [27]:
%cd /content/gdrive/My\ Drive/cpp_dl/example-app/build

/content/gdrive/My Drive/cpp_dl/example-app/build


##### The core Makefile is already generated.
##### Once you have made your changes able run this cell to compile and generate the actual binary "example-app"
##### The binary will be generated in the build/ folder.

In [38]:
!make

Scanning dependencies of target example-app
[-50%] Building CXX object CMakeFiles/example-app.dir/src/example-app.cpp.o
[  0%] Linking CXX executable example-app
[  0%] Built target example-app


#### Voila! Finally you get to run your model and save the world!

In [39]:
!./example-app

CUDA available! Training on GPU.
Initialize training and test dataloaders
Size of Training DataSet:60000
Size of Test DataSet:10000
Starting training+test cycle
Train Epoch: 1 [59008/60000] Loss: 0.2914
Test set: Average loss: 0.3289 | Accuracy: 0.867
Train Epoch: 2 [59008/60000] Loss: 0.2954
Test set: Average loss: 0.3000 | Accuracy: 0.875
Train Epoch: 3 [59008/60000] Loss: 0.2954
Test set: Average loss: 0.2953 | Accuracy: 0.876
Train Epoch: 4 [59008/60000] Loss: 0.0036
Test set: Average loss: 0.0367 | Accuracy: 0.989
Train Epoch: 5 [59008/60000] Loss: 0.0004
Test set: Average loss: 0.0441 | Accuracy: 0.988
Train Epoch: 6 [59008/60000] Loss: 0.0001
Test set: Average loss: 0.0380 | Accuracy: 0.990
Train Epoch: 7 [59008/60000] Loss: 0.0000
Test set: Average loss: 0.0378 | Accuracy: 0.991
Train Epoch: 8 [59008/60000] Loss: 0.0000
Test set: Average loss: 0.0323 | Accuracy: 0.991
Train Epoch: 9 [59008/60000] Loss: 0.0000
Test set: Average loss: 0.0348 | Accuracy: 0.991
Train Epoch: 10 [590